Dive Into：
-----
生成器是一类特殊的迭代器

In [50]:
class Fib:
    '''生成斐波拉契数列的Iterator'''
    
    def __init__(self, max):
        self.max = max # 赋值给实例变量
        
    def __iter__(self): # self类实例
        self.a = 0
        self.b = 1
        return self # 返回self
    
    def __next__(self): # self类实例
        fib = self.a
        if fib > self.max:
            raise StopIteration
        self.a, self.b = self.b, self.a + self.b
        return fib # 不要使用yield

### 类的定义
----1. __INIT__()方法

### 实例化类

In [22]:
# 创建一个Fib类的实例
fib = Fib(100)
# fib是Fib的实例
fib

In [14]:
# 每个类都具有一个内建属性__class__，它是该对象-实例所属的类
fib.__class__

__main__.Fib

In [12]:
fib.__doc__

'生成斐波拉契数列的Iterator'

### 实例变量
---
上述代码中的self.max，和作为参数传入__init__()方法的max是两回事。

事实上，self.max是实例内“全局”的，也就是可以在其他方法中访问它。

实例变量特定于某个类的实例，创建F两个具有不同max的实例，各自实例会记住自己的值，如下代码。

In [24]:
fib1 = Fib(180)
fib2 = Fib(200)
print(fib1.max)
print(fib2.max)

180
200


### Fibonacci Iterator
----
迭代器就是一个定义了__iter__()方法的类，这些类所有三种方法如上类（注意不是函数）的定义。

1. 从无到有创建一个迭代器， fib 应是一个类，而不是一个函 数。
2. “调用” Fib(max) 会创建该类一个真实的实例，并以 max 做为 参数调用__init__() 方法。 __init__() 方法以实例变量保存最 大值，以便随后的其他方法可以引用。
3. 当有人调用 iter(fib)的时候，__iter__()就会被调用。(正 如你等下会看到的， for 循环会自动调用它， 你也可以自己手 动调用。) 在完成迭代器初始化后，(在本例中， 重置我们两 个计数器 self.a 和 self.b)， __iter__() 方法能返回任何实 现了 __next__() 方法的对象。 在本例(甚至大多数例子)中， __iter__() 仅简单返回 self， 因为该类实现了自己的 __next__() 方法。
4. 当有人在迭代器的实例中调用 next()方法时，__next__() 会 自动调用。 随后会有更多理解。
5. 当 __next__() 方法抛出 StopIteration 异常， 这是给调用者 表示迭代用完了的信号。 和大多数异常不同， 这不是错误;它 是正常情况，仅表示迭代器没有值可产生了。 如果调用者是 for 循环， 它会注意到该 StopIteration 异常并优雅的退出。 (换句话说，它会吞掉该异常。) 这点神奇之处就是使用 for 的关键。
6. 为了分离出下一个值， 迭代器的 __next__() 方法简单 return 该值。 不要使用 yield ; 该语法上的小甜头仅用于你使 用生成器的时候。 这里你从无到有创建迭代器，使用 return 代 替。

完全晕了? 太好了。 让我们看如何调用该迭代器:

In [27]:
for n in Fib(1000):
    print(n, end=' ')

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 

1. 如你所见，for 循环调用 Fib(1000)————返回 Fib 类的实例。 这里叫作fib_inst。
2. for 循环调用 iter(fib_inst)， 它 返回迭代器。 叫它 fib_iter。 本例中， fib_iter == fib_inst， 因为 __iter__() 方法返回 self，但 for 循环不知道 (也不关心)那些。
3. 为“循环通过”迭代器， for 循环调用 next(fib_iter)， 它又调用 fib_iter 对象的 __next__() 方法，产生下一个Fibonacci计算并返回值。 for拿到该值并赋给 n， 然后执行 n 值的 for 循环 体。
4. for 循环如何知道什么时候结束?很高兴你问到。当 next(fib_iter) 抛出 StopIteration 异常时， for 循环将吞下 该异常并优雅退出。 (其他异常将传过并如常抛出。) 回想一下你在哪里见过 StopIteration 异常?当然是在 __next__() 方法里。

### Plural Rules Iterator
---
Rewrite our plural rules iterator

In [89]:
class LazyRules:
    '''实现了__iter__()和__iter__()的类，故可以被用作迭代器'''
    
    rules_filename = 'plural-rules.txt' # 类变量————生成的所有实例共享该变量
    
    def __init__(self):
        self.pattern_file = open(self.rules_filename, encoding = 'utf-8')
        self.cache = []
        
    def __iter__(self): # for循环调用iter(rules)时，__iter__()方法都会被调用
        self.cache_index = 0 # 初始化缓存，随后__next__()方法会用到它
        return self
    
    def __next__(self):
        self.cache_index += 1 # 记录缓存条目
        if len(self.cache) >= self.cache_index:
            return self.cache[self.cache_index - 1]
        
        if self.pattern_file.closed:
            raise StopIteration
            
        line = self.pattern_file.readline() # 从一个打开文件中读取下一行，文件对象是一个Iterator
        if not line: # 空行也会是一个字符'\n'，line如果是空串意味文件到头
            self.pattern_file.close() # 读取完所有内容，关闭文件
            raise StopIteration # 为for循环停止，迭代器结束
            
        pattern, search, replace = line.split(None, 3) # 随意多个空白分割，取3个
        funcs = build_match_and_apply_functions(pattern, search, replace)
        self.cache.append(funcs)
        return funcs
    
rules = LazyRules() # 实例化该类时，只是打开文件，不读区任何东西（随后进行）

In [90]:
# 更改类属性说明【Optimal】【Run only once】
r1 = LazyRules()
r2 = LazyRules()

# 类的每个实例继承了类属性和其在类中定义的值
print(r1.rules_filename)
print(r2.rules_filename)

print('------------')

# 即使修改一个实例属性的值也不会影响其他实例......也不会修改类变量(属性)
r2.rules_filename = 'r2-override.txt'
print(r2.rules_filename)
print(r1.rules_filename)

print('------------')

# 如果直接修改类属性，所有仍然继承该实例值的实例（如r1）会受影响??
r2.__class__.rules_filename = 'change.txt'
print(r1.rules_filename)

# 但已经覆盖（overridden）了该属性（如r2）但所有实例不受影响
print(r2.rules_filename)

plural-rules.txt
plural-rules.txt
------------
r2-override.txt
plural-rules.txt
------------
change.txt
r2-override.txt


1. 当模块引入时，创建了 LazyRules 类的一个单一实例， 叫 rules， 它打开模式文件但并没有读取。
2. 当要求第一个匹配和应用功能时，检查缓存并发现缓存为 空。 于是，从模式文件读取一行， 从模式中创建匹配和应用功 能，并缓存之。
3. 假如，因为参数的缘故，正好是第一行匹配了。如果那样，
4. 不会有更多的匹配和应用会创建，也不会有更多的行会从模式文件中读取。
5. 更进一步， 因为参数的缘故，假设调用者再次调用 plural() 函数来让一个不同的单词变复数。 plural() 函数中的 for 循环 会调用 iter(rules)，这会重置缓存索引但不会重置打开的文件 对象。
6. 第一次遍历， for 循环会从 rules 中索要一个值，该值会调 用其__next__()方法。然而这一次， 缓存已经被装入了一个匹 配和应用功能对， 与模式文件中第一行模式一致。 由于对前一 个单词做复数变换时已经被创建和缓存，它们被从缓存中返 回。 缓存索引递增，打开的文件无需访问。
7. 假如，因为参数的缘故，这一轮第一个规则 不 匹配。 所以 for 循环再次运转并从 rules 请求一个值。 这会再次调用 __next__() 方法。 这一次， 缓存被用完了——它仅有一个条 目，而我们被请求第二个——于是 __next__() 方法继续。 从打 开的文件中读取下一行，从模式中创建匹配和应用功能，并缓 存之。
8. 该“读取创建并缓存”过程一直持续直到我们从模式文件中读 取的规则与我们想变复数的单词不匹配。 如果我们确实在文件 结束前找到了一个匹配规则，我们仅需使用它并停止，文件还 一直打开。文件指针会留在我们停止读取，等待下一个 readline() 命令的地方。现在，缓存已经有更多条目了，并且 再次从头开始来将一个新单词变复数，在读取模式文件下一行 之前，缓存中的每一个条目都将被尝试。

##### 我们已达到“最佳要求”？
-------
1. 最小化初始代价。 

在 import 时发生的唯一的事就是实例化 一个单一的类并打开一个文件(但并不读取)。
2. 最大化性能

前述示例会在每次你想让一个单词变复数时，读 遍文件并动态创建功能。本版本将在创建的同时缓存功能，在 最坏情况下，仅需要读完一遍文件，无论你要让多少单词变复 数。

3. 将代码和数据分离。 

所有模式被存在一个分开的文件。代码 是代码，数据是数据，二者永远不会交织。


##### 还有一些优化（not seen）
-------
1. 这里有一些 LazyRules 示例需要细想的地 方: 

    模式文件被打开(在 __init__() 中)，并持续打开直到读取最后一个规 则。 当 Python 退出或最后一个 LazyRules 类的实例销毁，Python 会最终关闭文件， 但是那仍然可能会是一个很长的时间。如 果该类是一个“长时间运行”的 Python 进程 的一部分，Python 可能从不退出， LazyRules 对象就可能一直不会释放。

2； 这种情况有解决办法。 

    不要在 __init__() 中打开文件并让其在一行一行读取规则时 一直打开，你可以打开文件，读取所有规 则，并立即关闭文件。或你可以打开文 件，读取一条规则，用tell() 方法保存文件位置，关闭文件，后面再次打开它，使 用seek() 方法 继续从你离开的地方读取。 或者你不需担心这些就让文件打开，如同 本示例所做。 编程即是设计， 而设计牵扯 到所有的权衡和限制。让一个文件一直打 开太长时间可能是问题;让你代码太复杂 也可能是问题。哪一个是更大的问题，依 赖于你的开发团队，你的应用，和你的运 行环境。

##### （继续深入阅读）

见书的章节附录